In [80]:
import quandl
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import datetime
import dateutil

quandl.ApiConfig.api_key = 'bNhmL6hk7a3boYd7Rbfz'


def get_all_ndq():
    '''Get all security tickers!'''
    from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
    symbols = get_nasdaq_symbols()
    symbols = symbols.reset_index()

    symbols = symbols[symbols['Nasdaq Traded'] == True]
    return symbols['Symbol'].tolist()


def all_ndq_df():
    lst = get_all_ndq()
    if len(lst) < 8000:
        print(f'Error: Only {len(lst)} symbols called from get_all_nqd... check code!')
    data = quandl.get_table('SHARADAR/SF1',calendardate='2018-12-31',ticker=slst)
    return data

'''Daily data!'''
data = quandl.get_table("SHARADAR/DAILY", paginate=True)
tic_unique = data.ticker.unique()
data.tail()
print('Tickers:',tic_unique)

Tickers: ['XOM' 'WMT' 'VZ' 'UTX' 'UNH' 'TSLA' 'TRV' 'PG' 'PFE' 'NKE' 'MSFT' 'MRK'
 'MMM' 'MCD' 'KO' 'JPM' 'JNJ' 'INTC' 'IBM' 'HD' 'GS' 'GE' 'DD' 'CVX'
 'CSCO' 'CAT' 'BA' 'AXP' 'AAPL']


In [49]:
data.isnull().sum()
data.info()
data1 = data
data1.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378 entries, 0 to 2377
Data columns (total 10 columns):
ticker         2378 non-null object
date           2378 non-null datetime64[ns]
lastupdated    2378 non-null datetime64[ns]
ev             2378 non-null float64
evebit         2378 non-null float64
evebitda       2378 non-null float64
marketcap      2378 non-null float64
pb             2378 non-null float64
pe             2378 non-null float64
ps             2378 non-null float64
dtypes: datetime64[ns](2), float64(7), object(1)
memory usage: 185.9+ KB


,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps
None,,,,,,,,,,
2373,AAPL,2018-09-10,2018-10-21,1137146.7,16.2,14.0,1054517.7,9.2,18.8,4.1
2374,AAPL,2018-09-07,2018-10-21,1151491.6,16.4,14.2,1068862.6,9.3,19.0,4.2
2375,AAPL,2018-09-06,2018-10-21,1160185.5,16.5,14.3,1077556.5,9.4,19.2,4.2
2376,AAPL,2018-09-05,2018-10-21,1178394.3,16.7,14.5,1095765.3,9.5,19.5,4.3
2377,AAPL,2018-09-04,2018-10-21,1185590.9,16.8,14.6,1102961.9,9.6,19.7,4.3


In [50]:
from sklearn.impute import SimpleImputer
data.drop(columns=['ticker'])
## For the rest of the ratios, we will impute the missing values with the mean of the total column
imp=SimpleImputer(missing_values=np.nan, strategy="mean" )

#database1['ptb'] = imp.fit_transform(database1[['ptb']])
impute_columns=data.columns[data.isna().any()].tolist()
for i in impute_columns:
    data[i] = imp.fit_transform(data[[i]])
data['ticker']=data1.ticker

In [51]:

tickers = data.ticker.unique()

for ticker in tickers:
    try:
        # Get historical monthly prices
        #df = pdr.data.get_data_yahoo(ticker, start_date,interval='d').drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
        df = pdr.DataReader(ticker,'yahoo','2018-09-03','2018-12-31').drop(columns=['Open','High','Low','Close','Volume'])
        # Add pct change
        df['trt1m']=df['Adj Close'].pct_change()
        # Drop Adj Close column
        df.drop(columns=['Adj Close'], inplace=True)
        # Add tic column (to match complete)
        df['ticker'] = ticker
        df.reset_index(inplace=True)
        df = df.rename(columns={'Date':'date'})
        #df.rename(columns={'Date':'data_date'}, inplace=True)
        # Adjust data_date to be the end of the month(match complete)
        #df['data_date'] = df['data_date']-timedelta(days=1)
        #df['data_date'] = df['data_date']+timedelta(days=(1*365/12))
        #df['data_date'] = df['data_date']-timedelta(hours=10) # not sure why the timestamps have 10am on them?
        #type(df['data_date'].iloc[0])
        print(df.head(1))
        
    except:
        continue
    
    # Add df to complete via merge on columns
    data = pd.merge(data, df, how='outer', on=['date','ticker'])
    
    if 'trt1m_x' in data and 'trt1m_y' in data:
        # Combine x and y into single column
        data['trt1m'] = data['trt1m_x'].fillna(data['trt1m_y'])
        # Drop x and y columns
        data.drop(columns=['trt1m_x', 'trt1m_y'], inplace=True)
    
# Output new complete df
data

        date  trt1m ticker
0 2018-09-04    NaN    XOM
        date  trt1m ticker
0 2018-09-04    NaN    WMT
        date  trt1m ticker
0 2018-09-04    NaN     VZ
        date  trt1m ticker
0 2018-09-04    NaN    UTX
        date  trt1m ticker
0 2018-09-04    NaN    UNH
        date  trt1m ticker
0 2018-09-04    NaN   TSLA
        date  trt1m ticker
0 2018-09-04    NaN    TRV
        date  trt1m ticker
0 2018-09-04    NaN     PG
        date  trt1m ticker
0 2018-09-04    NaN    PFE
        date  trt1m ticker
0 2018-09-04    NaN    NKE
        date  trt1m ticker
0 2018-09-04    NaN   MSFT
        date  trt1m ticker
0 2018-09-04    NaN    MRK
        date  trt1m ticker
0 2018-09-04    NaN    MMM
        date  trt1m ticker
0 2018-09-04    NaN    MCD
        date  trt1m ticker
0 2018-09-04    NaN     KO
        date  trt1m ticker
0 2018-09-04    NaN    JPM
        date  trt1m ticker
0 2018-09-04    NaN    JNJ
        date  trt1m ticker
0 2018-09-04    NaN   INTC
        date  trt1m ticker
0

,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,trt1m
0,XOM,2018-12-31,2018-12-31,323071.3,12.3,7.1,288703.3,1.5,12.4,1.1,0.000294
1,XOM,2018-12-28,2018-12-28,322986.6,12.3,7.1,288618.6,1.5,12.4,1.1,-0.011169
2,XOM,2018-12-27,2018-12-27,326246.7,12.5,7.1,291878.7,1.5,12.6,1.1,0.004370
3,XOM,2018-12-26,2018-12-26,324976.5,12.4,7.1,290608.5,1.5,12.5,1.1,0.047779
4,XOM,2018-12-24,2018-12-24,311724.7,11.9,6.8,277356.7,1.5,11.9,1.0,-0.038315
...,...,...,...,...,...,...,...,...,...,...,...
2373,AAPL,2018-09-10,2018-10-21,1137146.7,16.2,14.0,1054517.7,9.2,18.8,4.1,-0.013421
2374,AAPL,2018-09-07,2018-10-21,1151491.6,16.4,14.2,1068862.6,9.3,19.0,4.2,-0.008068
2375,AAPL,2018-09-06,2018-10-21,1160185.5,16.5,14.3,1077556.5,9.4,19.2,4.2,-0.016617
2376,AAPL,2018-09-05,2018-10-21,1178394.3,16.7,14.5,1095765.3,9.5,19.5,4.3,-0.006525


In [56]:
data.loc[data['ticker'] =='AAPL'].tail()

,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,trt1m
2373,AAPL,2018-09-10,2018-10-21,1137146.7,16.2,14.0,1054517.7,9.2,18.8,4.1,-0.013421
2374,AAPL,2018-09-07,2018-10-21,1151491.6,16.4,14.2,1068862.6,9.3,19.0,4.2,-0.008068
2375,AAPL,2018-09-06,2018-10-21,1160185.5,16.5,14.3,1077556.5,9.4,19.2,4.2,-0.016617
2376,AAPL,2018-09-05,2018-10-21,1178394.3,16.7,14.5,1095765.3,9.5,19.5,4.3,-0.006525
2377,AAPL,2018-09-04,2018-10-21,1185590.9,16.8,14.6,1102961.9,9.6,19.7,4.3,NaN


In [60]:
'''Merge in SP500 -- + Win col'''

#SP500= pd.read_csv('^GSPC.csv',usecols=["Date", "Adj Close"]) # AE commented out
SP500= pd.read_csv('^GSPC.csv',usecols=["Date", "Adj Close"]) # changed file location

SP500['sprtrn']= SP500['Adj Close'].pct_change() #*100     ## Creates the column sprtrn with the monhtly SP500 return
SP500= SP500[np.isfinite(SP500['sprtrn'])]
SP500.Date=pd.to_datetime(SP500.Date)


# AE changed datadate to data_date
complete = pd.merge(data.assign(grouper=data['date']),
               SP500.assign(grouper=SP500['Date']),
               how='left', on='grouper')

complete=complete.drop(columns=['grouper', 'Date', 'Adj Close']) # Adj close for SP500

# drop duplicate columns
complete = complete.loc[:,~complete.columns.duplicated()]

complete

,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,trt1m,sprtrn
0,XOM,2018-12-31,2018-12-31,323071.3,12.3,7.1,288703.3,1.5,12.4,1.1,0.000294,0.008492
1,XOM,2018-12-28,2018-12-28,322986.6,12.3,7.1,288618.6,1.5,12.4,1.1,-0.011169,-0.001242
2,XOM,2018-12-27,2018-12-27,326246.7,12.5,7.1,291878.7,1.5,12.6,1.1,0.004370,0.008563
3,XOM,2018-12-26,2018-12-26,324976.5,12.4,7.1,290608.5,1.5,12.5,1.1,0.047779,0.049594
4,XOM,2018-12-24,2018-12-24,311724.7,11.9,6.8,277356.7,1.5,11.9,1.0,-0.038315,-0.027112
...,...,...,...,...,...,...,...,...,...,...,...,...
2373,AAPL,2018-09-10,2018-10-21,1137146.7,16.2,14.0,1054517.7,9.2,18.8,4.1,-0.013421,0.001898
2374,AAPL,2018-09-07,2018-10-21,1151491.6,16.4,14.2,1068862.6,9.3,19.0,4.2,-0.008068,-0.002213
2375,AAPL,2018-09-06,2018-10-21,1160185.5,16.5,14.3,1077556.5,9.4,19.2,4.2,-0.016617,-0.003652
2376,AAPL,2018-09-05,2018-10-21,1178394.3,16.7,14.5,1095765.3,9.5,19.5,4.3,-0.006525,-0.002803


In [70]:
'''Create win column'''

complete['win'] = np.where(complete['trt1m'] > complete['sprtrn'],1,0)

In [ ]:
'''

Feature selection -- not used in daily


NO NEED FOR THIS IN DAILY -- SKIP DOWN

'''

train = clean_db.drop(columns = ['win','conm','datadate','gvkey','trt1m','decision_date','tic'])
train_labels = clean_db['win']
fs = FeatureSelector(data = train, labels = train_labels)



In [ ]:
fs.identify_collinear(correlation_threshold=0.975)
fs.plot_collinear(plot_all=True)

In [ ]:
fs.identify_zero_importance(task = 'regression', eval_metric = 'auc', 
                            n_iterations = 10, early_stopping = True)

In [ ]:
fs.identify_low_importance(cumulative_importance = 0.99)

In [ ]:
all_to_remove = fs.check_removal()
print(all_to_remove)
clean_removed_db = clean_db.drop(columns = all_to_remove)
clean_removed_db.drop_duplicates(keep=False,inplace=True) 

In [74]:
comp2 = complete
clean_db = complete
clean_db.head(1)

,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,trt1m,sprtrn,win
0,XOM,2018-12-31,2018-12-31,323071.3,12.3,7.1,288703.3,1.5,12.4,1.1,0.000294,0.008492,0


In [99]:
def rfstockpicker_backtest(database,prediction_date,rolling_window,nb_wks,n_estimators,n_jobs):
    '''
    Modified to be weekly windows, predict daily - weekly
    '''
    database = database
    results={}
    pd.options.mode.chained_assignment = None # Gets rid of a warning
    for i in range(0,nb_wks):
        d = datetime.datetime.strptime(prediction_date, "%Y-%m-%d")
        
        d2 = d - dateutil.relativedelta.relativedelta(weeks=i)
        d3 = d - dateutil.relativedelta.relativedelta(weeks=i+1)
        d4 = d3 - dateutil.relativedelta.relativedelta(weeks=rolling_window)    ##Creating a rolling window interval
        
        test_set = database[database['date'].isin(pd.date_range(d2, d2))]
        training_set=database[database['date'].isin(pd.date_range(d4, d3))]

        # Labels are the values we want to predict
        train_labels = np.array(training_set['win'])
        # Remove the labels from the features
        columns_to_drop=['win','lastupdated','sprtrn','trt1m','date','ticker']
        train_features= training_set.drop(columns_to_drop, axis = 1)
        # Saving feature names for later use
        train_feature_list = list(train_features.columns)
        # Convert to numpy array
        train_features = np.array(train_features)
        
        test_labels = np.array(test_set['win'])
        # Remove the labels from the test features
        test_features= test_set.drop(columns_to_drop, axis = 1)
        # Convert to numpy array
        test_features = np.array(test_features)

        # Import the model we are using
        from sklearn.ensemble import RandomForestRegressor
        # Instantiate model 
        rf = RandomForestRegressor(n_estimators = n_estimators, random_state = 42,n_jobs=n_jobs)
        # Train the model on training data
        rf.fit(train_features, train_labels);

        # Use the forest's predict method on the test data
        stock_predictions = rf.predict(test_features)
        result_set=test_set
        result_set['predictions']=stock_predictions 
        results[i]=result_set

        '''
        Not part of the daily model...
       # Get numerical feature importances
        importances = list(rf.feature_importances_)
        # List of tuples with variable and importance
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(train_feature_list, importances)]
        # Sort the feature importances by most important first
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
        '''
    return results,train_feature_list #feature_importances,importances

In [109]:
results,features = rfstockpicker_backtest(clean_db,("2018-12-31"),7,10,100,24)


In [103]:
results[0].nlargest(10, 'predictions')
res = results[0].nlargest(10, 'predictions')

,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,trt1m,sprtrn,win,predictions
1148,KO,2018-12-31,2018-12-31,232590.9,25.6,22.8,201545.9,11.0,71.7,6.2,0.003178,0.008492,0,0.950000
1066,MCD,2018-12-31,2018-12-31,166211.2,18.7,16.0,136890.5,-20.2,26.3,6.5,0.011449,0.008492,1,0.830000
1230,JPM,2018-12-31,2019-02-26,421896.6,11.0,9.2,324626.6,1.3,11.7,3.0,0.008159,0.008492,0,0.800000
164,VZ,2018-12-31,2018-12-31,342705.9,13.4,8.0,232301.9,4.3,7.2,1.8,0.017188,0.008492,1,0.775833
656,PFE,2018-12-31,2018-12-31,289795.7,21.1,14.4,252317.7,3.5,10.6,4.7,0.016062,0.008492,1,0.760000
0,XOM,2018-12-31,2018-12-31,323071.3,12.3,7.1,288703.3,1.5,12.4,1.1,0.000294,0.008492,0,0.710000
410,TSLA,2018-12-31,2019-03-25,65805.7,-58.3,88.4,57152.7,12.7,-31.9,3.3,-0.003205,0.008492,0,0.690000
738,NKE,2018-12-31,2018-12-31,117946.4,25.8,22.2,117742.4,13.1,56.7,3.2,0.010908,0.008492,1,0.680000
328,UNH,2018-12-31,2018-12-31,262952.0,16.0,14.0,239662.0,4.8,19.1,1.1,0.011696,0.008492,1,0.550000
902,MRK,2018-12-31,2018-12-31,214460.8,27.0,17.2,198694.8,6.1,59.4,4.8,0.013798,0.008492,1,0.550000


In [110]:
monthly_yields=[]
SP500_monthly_yields=[]
compounded_portfolio=1
compounded_SP500=1
for i in range(0,len(results)):
    monthly_yields.append((results[i].nlargest(5, 'predictions')).trt1m.mean())
    SP500_monthly_yields.append(results[i].sprtrn.mean())
    compounded_portfolio=(compounded_portfolio*(1+(monthly_yields[i]/100)))
    compounded_SP500=compounded_SP500*(1+(SP500_monthly_yields[i]/100))
print('The compounded return of the portfolio over the period is:',round(((compounded_portfolio-1))*100,2),'%')
print('The compounded return of the SP500 over the period is:',round((compounded_SP500-1)*100,2),'%')

The compounded return of the portfolio over the period is: -0.01 %
The compounded return of the SP500 over the period is: -0.05 %


In [127]:
excess = ((compounded_portfolio-1) - (compounded_SP500-1))*100
ann_excess = (1+excess)**52
print(f'Annualized excess returns should be on average {round(ann_excess,2)}% above SPY')

Annualized excess returns should be on average 8.86% above SPY


In [187]:
'''
Resample not working... uhh fuck resample -- lets do this by hand.
spy = pdr.DataReader('SPY','yahoo').drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
spy = spy.pct_change()
spy.resample('Y').mean()

spy
'''


"spy = pdr.DataReader('SPY','yahoo').drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])\nspy = spy.pct_change()\nspy.resample('Y').mean()\n\nspy\n"

In [2]:
import yfinance as yf
#'2005-01-03','2006-01-03','2007-01-03','2008-01-03','2009-01-03',
dates = ['2010-01-03','2011-01-03','2012-01-03','2013-01-03','2014-01-03','2015-01-03']

s = yf.download("SPY", start='2000-01-01', end="2017-04-30",interal='1y').drop(columns=['Open','High','Low','Close','Volume'])
s.reset_index(inplace=True)
#s.loc[s['Date'].str.endswith('01-03'),:]

spy_ann = s.loc[s['Date'].isin(dates),['Adj Close']]
spy_avg = spy_ann.pct_change().mean().values

[*********************100%***********************]  1 of 1 completed


In [202]:
print(F'Average YoY return (cumulative) of {round((spy_avg[0]*100)+ann_excess,4)} %')

Average YoY return (cumulative) of 24.6357 %


In [219]:
print('Confirming return calculations, and trying monthly')
ko = (46.5 - 45.9)/45.9
mcd = (174.57 - 173.39)/173.39
jpm = (99.73 - 94.06)/94.06
vz = (52.28 - 53.38)/53.38
vzw = 55.09 - 53.52
pfe = (40.93 - 41.04)/41.04
xom = (46.08 - 43.44)/43.44
tsla = (365 - 357)/357
tsla = (320-307)/307
unh = (261-265)/265
nke = (84.8 - 81.02)/81.02
mrk = (79.2 - 72.1)/72.1

tot = ko + mcd + jpm + vz + pfe + xom + tsla + unh + nke + mrk
ann = (tot*100)
print(f'Manually calculated monthly return -- {round(ann,4)} %')
print(f'Annualized -- {((tot+1) * 100)**12} %')

Confirming return calculations, and trying monthly
Manually calculated monthly return -- 29.0025 %
Annualized -- 2.1241058139163784e+25 %


In [294]:
'''Select top 5!!'''

results[0].nlargest(10, 'predictions')
res = results[0].nlargest(10, 'predictions')
top5 = res.trt1m.head(5).mean()
#res.sprtrn[0]
anntop5 = (top5 + 1)**52
anntop5
print(f'Top 5 picks -- annualized return {round(anntop5,2)*100} %')

Top 5 picks -- annualized return 179.0 %


In [225]:
#res = df.groupby(pd.PeriodIndex(df.columns, freq='Q'), axis=0).mean()
#.rename(columns=lambda c: str(c).lower()))
type(s.Date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [401]:
s
df = s.set_index('Date').asfreq('Q').reset_index().dropna()
#c = df.columns.difference(['Adj Close'])
#df[c] = df[c].ffill()
#df['Type'] = df['Adj Close'].fillna('')
df.head()

,Date,Adj Close
0,2000-03-31,103.635895
1,2000-06-30,100.361099
7,2001-12-31,80.415688
10,2002-09-30,58.169739
11,2002-12-31,63.058281


In [6]:


tics = ['AAPL','AMZN']
import pandas_datareader as pdr

for ticker in tics:
    #df = pdr.data.get_data_yahoo(ticker,'2014-01-01',interval='m').drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    #df = pdr.DataReader(ticker,'yahoo','2014-01-01').drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    df = yf.download(ticker, start='2014-01-01', end="2018-09-30").drop(columns=['Open','High','Low','Close','Volume'])
    df['trt1m']=df['Adj Close'].pct_change()
    df.drop(columns=['Adj Close'], inplace=True)
    df['tic'] = ticker
    df.reset_index(inplace=True)
    df = df.set_index('Date').asfreq('Q').reset_index()#.dropna() #New! (Moved down) ---

    #df.rename(columns={'Date':'data_date'}, inplace=True)
    #df['data_date'] = df['data_date']-pd.Timedelta(days=1)
    #df['data_date'] = df['data_date']+pd.Timedelta(days=(1*365/12))
    #df['data_date'] = df['data_date']-pd.Timedelta(hours=10)
              
    #try:   
        # Adjust data_date to be the end of the month(match complete)
        #df['data_date'] = df['data_date']-timedelta(days=1)
        #df['data_date'] = df['data_date']+timedelta(days=(1*365/12))
        #df['data_date'] = df['data_date']-timedelta(hours=10) # not sure why the timestamps have 10am on them?
        #type(df['data_date'].iloc[0])
              
    #except:
        #continue
        
    # Add df to complete via merge on columns
    #complete = pd.merge(complete, df, how='outer', on=['tic', 'data_date'])
    
    #if 'trt1m_x' in complete and 'trt1m_y' in complete:
        # Combine x and y into single column
        #complete['trt1m'] = complete['trt1m_x'].fillna(complete['trt1m_y'])
        # Drop x and y columns
        #complete.drop(columns=['trt1m_x', 'trt1m_y'], inplace=True)
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,trt1m,tic
0,2014-03-31,-0.005676,AMZN
1,2014-06-30,0.000647,AMZN
2,2014-09-30,0.001927,AMZN
3,2014-12-31,0.000161,AMZN
4,2015-03-31,-0.006647,AMZN
5,2015-06-30,0.009840,AMZN
6,2015-09-30,0.031891,AMZN
7,2015-12-31,-0.019127,AMZN
8,2016-03-31,-0.008435,AMZN
9,2016-06-30,0.000028,AMZN
